# 11장 자연어처리 1부

## 11.1 자연어처리 소개

## 11.2 텍스트 벡터화

## 11.3 단어 모음 표현법: 집합과 시퀀스

여기서는 IMDB 영화 후기 데이터를 이용하여 두 모델 방식의 
활용법과 차이점을 소개한다.

### 11.3.1 IMDB 영화 후기 데이터 준비

이전과는 달리 여기서는 IMDB 데이터셋을 직접 다운로드하여 전처리하는 
과정을 살펴본다. 

준비 과정 1: 데이터셋 다운로드 압축 풀기

압축을 풀면 아래 구조의 디렉토리가 생성된다.

```
aclImdb/
...train/
......pos/
......neg/
...test/
......pos/
......neg/
```

`train`의 `pos`와 `neg` 서브디렉토리에 각각 12,500개의 긍정과 부정 후기가
포함되어 있다. 

In [1]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gzs

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  9232k      0  0:00:08  0:00:08 --:--:-- 16.3M


`aclImdb/train/unsup` 서브디렉토리는 필요 없기에 삭제한다.

In [2]:
if 'google.colab' in str(get_ipython()):
    !rm -r aclImdb/train/unsup
else: 
    import shutil
    unsup_path = './aclImdb/train/unsup'
    shutil.rmtree(unsup_path)

긍정 후기 하나의 내용을 살펴보자.
모델 구성 이전에 훈련 데이터셋을 살펴 보고
모델에 대한 직관을 갖는 과정이 항상 필요하다.

In [3]:
if 'google.colab' in str(get_ipython()):
    !cat aclImdb/train/pos/4077_10.txt
else:
    with open('aclImdb/train/pos/4077_10.txt', 'r') as f:
        text = f.read()
        print(text)

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

준비 과정 2: 검증셋 준비

훈련셋의 20%를 검증셋으로 떼어낸다.
이를 위해 `aclImdb/val` 디렉토리를 생성한 후에
긍정과 부정 훈련셋 모두 무작위로 섞은 후 그중 20%를 검증셋 디렉토리로 옮긴다.

In [4]:
import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

for category in ("neg", "pos"):
    os.makedirs(val_dir / category)            # val 디렉토리 생성
    files = os.listdir(train_dir / category)
    
    random.Random(1337).shuffle(files)         # 훈련셋 무작위 섞기
    
    num_val_samples = int(0.2 * len(files))    # 20% 지정 후 검증셋으로 옮기기
    val_files = files[-num_val_samples:]
    
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

준비 과정 3: 텐서 데이터셋 준비

`text_dataset_from_directory()` 함수를 이용하여 
훈련셋, 검증셋, 테스트셋을 준비한다. 
자료형은 모두 `Dataset`이며, 배치 크기는 32를 사용한다.

In [5]:
from tensorflow import keras

batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
    )

val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
    )

test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
    )

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


각 데이터셋은 배치로 구분되며
입력은 `tf.string` 텐서이고, 타깃은 `int32` 텐서이다.
크기는 모두 32이며 지정된 배치 크기이다.
예를 들어, 첫째 배치의 입력과 타깃 데이터의 정보는 다음과 같다.

In [6]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    
    # 예제: 첫째 배치의 첫째 후기
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    
    break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b'****SPOILERS**** The film "Sniper" is undoubtedly based on the exploits of legendary US Marine sniper Carlos "Gunny" Hathcock. The unassuming soft-spoken Mister Rogers look-alike who ran up a score of as much as 300 confirmed and unconfirmed Vietcong and North Vietnamese military kills during his two tours in "Nam".Which shows just how deadly and effective a trained military sniper really is.<br /><br />Tom Berenger is cool clam and deadly as Sgt.Thomas Beckett who\'s at the end of his career as a top US Marine sniper but who later in the movie realizes that a life as a civilian will be pointless. Since there\'s nothing outside for him to do with his skills that he learned in the US Marines unless he decides to become a mob hit-man. Backett reluctantly accepts his fate as a lifetime professional killer for his country.<br /><br />The story of the film "Sniper"

### 11.3.2 단어주머니 기법

**방식 1: 유니그램 멀티-핫 인코딩**

`TextVectorization` 클래스의 `output_mode="multi_hot"` 옵션을 이용한다.

In [7]:
from tensorflow.keras.layers import TextVectorization

text_vectorization = TextVectorization(
    max_tokens=20000,
    output_mode="multi_hot",
)

# 어휘색인 생성
text_only_train_ds = train_ds.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds)

생성된 어휘색인을 이용하여 훈련셋, 검증셋, 테스트셋 모두 벡터화한다. 

In [8]:
binary_1gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y))
binary_1gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y))
binary_1gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y))

변환된 첫째 배치의 입력과 타깃 데이터의 정보는 다음과 같다.
`max_tokens=20000`으로 지정하였기에 모든 문장은 길이가 2만인 벡터로 변환되었다.

In [9]:
for inputs, targets in binary_1gram_train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32, 20000)
inputs.dtype: <dtype: 'float32'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
targets[0]: tf.Tensor(1, shape=(), dtype=int32)


*밀집 모델 활용*

단어주머니 모델로 여기서는 밀집 모델을 사용한다. 
`get_model()` 함수가 컴파일 된 단순한 밀집 모델을 반환한다.
모델의 출력값은 긍정일 확률이며, 
최상위 층의 활성화 함수로 `sigmoid`를 사용한다.

In [10]:
from tensorflow import keras
from tensorflow.keras import layers

def get_model(max_tokens=20000, hidden_dim=16):
    inputs = keras.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation="relu")(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)  # 긍정일 확률 계산
    
    model = keras.Model(inputs, outputs)
    
    model.compile(optimizer="rmsprop",
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    
    return model

In [11]:
model = get_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________


밀집 모델 훈련과정은 특별한 게 없다.
훈련 후 테스트셋에 대한 정확도가 89% 보다 조금 낮게 나온다.
최고 성능의 모델이 테스트셋에 대해 95% 정도 정확도를 내는 것보다는 낮지만
무작위로 찍는 모델보다는 훨씬 좋은 모델이다.

In [12]:
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_1gram.keras",
                                    save_best_only=True)
]

model.fit(binary_1gram_train_ds.cache(),
          validation_data=binary_1gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)

model = keras.models.load_model("binary_1gram.keras")
print(f"Test acc: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

Epoch 1/10
625/625 [==============================] - 10s 12ms/step - loss: 0.4097 - accuracy: 0.8288 - val_loss: 0.2863 - val_accuracy: 0.8892
Epoch 2/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2772 - accuracy: 0.8979 - val_loss: 0.2845 - val_accuracy: 0.8942
Epoch 3/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2424 - accuracy: 0.9151 - val_loss: 0.2947 - val_accuracy: 0.8942
Epoch 4/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2307 - accuracy: 0.9214 - val_loss: 0.3101 - val_accuracy: 0.8910
Epoch 5/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2229 - accuracy: 0.9246 - val_loss: 0.3275 - val_accuracy: 0.8892
Epoch 6/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2197 - accuracy: 0.9274 - val_loss: 0.3413 - val_accuracy: 0.8874
Epoch 7/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2174 - accuracy: 0.9291 - val_loss: 0.3532 - val_accuracy: 0.8874
Epoc

**방식 2: 바이그램 멀티-핫 인코딩**

`TextVectorization` 클래스의 `ngrams=N` 옵션을 이용하면
N-그램들로 이루어진 어휘색인을 생성할 수 있다.

In [13]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="multi_hot",
)

어휘색인 생성과 훈련셋, 검증셋, 테스트셋의 벡터화 과정은 동일하다. 

In [14]:
text_vectorization.adapt(text_only_train_ds)

binary_2gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y))
binary_2gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y))
binary_2gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y))

훈련 후 테스트셋에 대한 정확도가 90%를 조금 웃돌 정도로 많이 향상되었다.

In [15]:
model = get_model()

callbacks = [
    keras.callbacks.ModelCheckpoint("binary_2gram.keras",
                                    save_best_only=True)
]

model.fit(binary_2gram_train_ds.cache(),
          validation_data=binary_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)

model = keras.models.load_model("binary_2gram.keras")
print(f"Test acc: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")

Epoch 1/10
625/625 [==============================] - 8s 12ms/step - loss: 0.3749 - accuracy: 0.8504 - val_loss: 0.2578 - val_accuracy: 0.9020
Epoch 2/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2433 - accuracy: 0.9147 - val_loss: 0.2616 - val_accuracy: 0.9052
Epoch 3/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2013 - accuracy: 0.9330 - val_loss: 0.2757 - val_accuracy: 0.9068
Epoch 4/10
625/625 [==============================] - 3s 4ms/step - loss: 0.1919 - accuracy: 0.9405 - val_loss: 0.2893 - val_accuracy: 0.9058
Epoch 5/10
625/625 [==============================] - 3s 4ms/step - loss: 0.1866 - accuracy: 0.9451 - val_loss: 0.3058 - val_accuracy: 0.9066
Epoch 6/10
625/625 [==============================] - 3s 4ms/step - loss: 0.1784 - accuracy: 0.9475 - val_loss: 0.3146 - val_accuracy: 0.9052
Epoch 7/10
625/625 [==============================] - 3s 4ms/step - loss: 0.1781 - accuracy: 0.9489 - val_loss: 0.3304 - val_accuracy: 0.9036
Epoch

**방식 3: 바이그램 TF-IDF 인코딩**

`output_mode="tf_idf"` 옵션을 사용하면 TF-IDF 인코딩을 지원한다.

In [16]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="tf_idf",
)

훈련 후 테스트셋에 대한 정확도가 다시 89% 아래로 내려간다.
여기서는 별 도움이 되지 않았지만 많은 텍스트 분류 모델에서는 1% 정도의 성능 향상을 가져온다.

In [17]:
text_vectorization.adapt(text_only_train_ds)

tfidf_2gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y))
tfidf_2gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y))
tfidf_2gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y))

model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("tfidf_2gram.keras",
                                    save_best_only=True)
]
model.fit(tfidf_2gram_train_ds.cache(),
          validation_data=tfidf_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)

model = keras.models.load_model("tfidf_2gram.keras")
print(f"Test acc: {model.evaluate(tfidf_2gram_test_ds)[1]:.3f}")

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_4 (Dense)             (None, 16)                320016    
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 9s 13ms/step - loss: 0.5317 - accuracy: 0.7610 - val_loss: 0.3231 - val_accuracy: 0.8602
Epoch 2/10
625/625 [======================

### 11.3.3 시퀀스 활용법

**정수 벡터 데이터셋 준비**

훈련셋의 모든 후기 문장을 정수들의 벡터로 변환한다.
단, 후기 문장이 최대 600개의 단어만 포함하도록 한다. 
또한 사용되는 어휘는 빈도 기준 최대 2만 개로 제한한다. 

- `max_length = 600`
- `max_tokens = 20000`
- `output_sequence_length=max_length`

In [18]:
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000

text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)

text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y))
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y))
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y))

변환된 첫째 배치의 입력과 타깃 데이터의 정보는 다음과 같다.
`output_sequence_length=600`으로 지정하였기에 모든 문장은 단어를 최대 600개에서
잘린다. 따라서 생성되는 정수들의 벡터는 길이가 모두 600으로 지정된다.
물론 문장이 600개보다 적은 수의 단어를 사용한다면 나머지는 0으로 채워진다. 
또한 벡터에 사용된 정수는 2만보다 작은 값이며, 
이는 빈도가 가장 높은 2만개의 단어만을 대상(`max_tokens=20000`)으로 했기 때문이다.

In [19]:
for inputs, targets in int_train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32, 600)
inputs.dtype: <dtype: 'int64'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(
[    1    37   226  4343   265    37 19719     6  2781   539     8    11
   957   929    18     2    20   182    39     9    14    91     8  3729
     6    32   144  2972   775    36  4330  2781   223    14  1626    65
   321   429    42   246    99   501   518   544     8    11    29     4
   354  7407  2781   329     7   259     6   215   142    37     2  2662
   745     1     6   325    46     3   132     2   113     7  2123     3
  5230     2     1    16   462    34   429  1453  6041   656   127     4
   481     1     2   795    24    49    94     2    18    38   632     3
    85     3     5   258   207   157    39 11520 16065   385     4  8887
   825  2075    49   572     6     2   115   612   382     4    18    16
   250     4   164   341   126    16     4    50   468     3   132   320
    31     9    94     9     4  1689   809    17 15135  1083  3493 

#### 11.3.3.1 원-핫 단어 벡터

아래 코드는 `tf.one_hot()` 함수를 전처리로 활용하는 순환 신경망 모델을 정의한다. 이렇게 하면 정수 벡터를 바로 모델의 입력값으로 사용할 수 있다.

In [20]:
import tensorflow as tf

inputs = keras.Input(shape=(None,), dtype="int64") # None 대신 600을 사용해도 됨

# 원-핫 인코딩
embedded = tf.one_hot(inputs, depth=max_tokens)  # (600, 20000) 모양의 출력값 생성

x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None)]            0         
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 20000)       0         
                                                                 
 bidirectional (Bidirectiona  (None, 64)               5128448   
 l)                                                              
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,128,513
Trainable params: 5,128,513
Non-trainable params: 0
_________________________________________________

모델 훈련이 매우 느리다. 
이유는 입력 데이터가 너무 많은 특성을 갖기 때문이다. 
입력 데이터 하나의 모양과 특성 수는 다음과 같다.

- 모양: `(600, 20000)`
- 특성 수: `600 * 20,000 = 12,000,000`

양방향 LSTM은 엄청난 양의 반복을 실행하기에 당연히 훈련 시간이 길어진다.
게다가 훈련된 모델의 성능이 별로 좋지 않다.
테스테셋에 대한 정확도가 87% 정도에 불과해서
바이그램 모델보다 성능이 낮다.

**주의사항**: 모델 훈련과정을 한 번 보기만 하려면 `epochs=1`로 설정하는 것을 권장한다.
책에서는 원래 `epochs=10`을 사용하였는데 컴퓨터 성능에 따라 몇 시간이 소요될 수 있다.

In [21]:
callbacks = [
    keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras",
                                    save_best_only=True)
]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=1, callbacks=callbacks)

model = keras.models.load_model("one_hot_bidir_lstm.keras")

print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

782/782 [==============================] - 104s 132ms/step - loss: 0.3697 - accuracy: 0.8661
Test acc: 0.866


#### 11.3.3.2. 단어 임베딩

아래 코드는 단어 임베딩을 모델 구성에 직접 활용하는 것을 보여준다.
여전히 양방향 LSTM 층을 사용한다.

In [22]:
inputs = keras.Input(shape=(None,), dtype="int64")  # None 대신 600을 사용해도 됨

# 단어 임베딩
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)

x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras",
                                    save_best_only=True)
]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)

model = keras.models.load_model("embeddings_bidir_gru.keras")

print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         5120000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               73984     
 nal)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
_________________________________________________

훈련은 원-핫 인코딩 방식보다 훨씬 빠르게 이루어지며 성능은 87% 정도로 비슷하다. 
바이그램 모델보다 성능이 여전히 떨어지는 이유 중에 하나는 후기에 사용된 단어의 수를 600개로 제한하였기 때문이다. 

**패딩과 마스킹**

아래 코드는 마스킹을 활용하는 방식을 보여준다.

- `mask_zero=True` 옵션: 마스킹 옵션 켜기

In [23]:
inputs = keras.Input(shape=(None,), dtype="int64") # None 대신 600을 사용해도 됨

# 마스킹 활용 단어 임베딩
embedded = layers.Embedding(
    input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)

x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru_with_masking.keras",
                                    save_best_only=True)
]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)

model = keras.models.load_model("embeddings_bidir_gru_with_masking.keras")

print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 64)               73984     
 nal)                                                            
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
_________________________________________________

모델 성능이 88% 정도로 살짝 향상된다.

#### 11.3.3.3. GloVe 단어 임베딩

[glove.6B.zip](http://nlp.stanford.edu/data/glove.6B.zip) 파일을 다운로드해서 압축을 푼다.
그러면 다음 세 개의 파일이 생성된다.

- `glove.6B.50d.txt` (약 1 GB)
- `glove.6B.100d.txt` (약 670 MB)
- `glove.6B.200d.txt` (약 330 MB)
- `glove.6B.300d.txt` (약 160 MB)

In [24]:
if 'google.colab' in str(get_ipython()):
    !wget http://nlp.stanford.edu/data/glove.6B.zip
    !unzip -q glove.6B.zip
else: 
    try: 
        import wget, zipfile
    except ModuleNotFoundError: 
        !pip install wget
        
    import wget, zipfile
    wget.download('http://nlp.stanford.edu/data/glove.6B.zip')
    with zipfile.ZipFile('glove.6B.zip', 'r') as zip_ref:
        zip_ref.extractall('./')

--2022-11-30 14:52:53--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-11-30 14:52:53--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-11-30 14:52:54--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

총 40만 개의 단어 정보를 포함한 `glove.6B.100d.txt` 파일을 이용하여 단어 임베딩을 실행하기 위해
먼저 파일 내용을 사전으로 가져온다.

- `embeddings_index`: 키(key)는 단어, 값(value)은 단어와 연관된 길이가 100인 벡터의 쌍으로
이뤄진 사전을 가리킨다.

In [25]:
import numpy as np
path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}

with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print(f"Found {len(embeddings_index)} word vectors.")

Found 400000 word vectors.


단어 임베딩에 사용될 (20000, 100) 모양의 행렬을 준비한다.
행렬의 각 행은 훈련셋 영화 후기에 포함된 단어 각각에 해당하는
길이가 100인 벡터로 구성도다.
각 단어와 연결된 벡터는 `embeddings_index`를 이용해서 구한다.

In [26]:
embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
    if i < max_tokens:  # 굳이 필요 없음. 이유는 len(vocabulary)가 2만이기 때문임.
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

`Embedding` 객체를 생성할 때 앞서 생성한 `embedding_matrix`를 
단어 임베딩의 기준으로 사용하도록 지정한다.

- `Constant`: 단어 임베딩의 기준으로 사용될 행렬을 상수로 지정한다.
- `trainable=False`: 단어 임베딩의 기준이 모델 훈련 과정 내내 고정시킨다. 
    즉, 단어 임베딩에 대한 별도 훈련은 시키지 않는다.

In [27]:
embedding_layer = layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
    mask_zero=True,
)

모델 구성은 임베딩 층만 제외하고 이전과 동일하다.
훈련 결과가 특별히 나아지지는 않는다. 이유는 훈련 데이터셋이 충분히 크기에 굳이 외부 임베딩 파일을
이용하지 않아도 자체적으로 단어들 사이의 관계를 잘 찾기 때문이다.
하지만 훈련셋이 더 작은 경우 잘 훈련된 임베딩 파일을 이용하면 보다 효율적으로 훈련이 진행된다.

In [28]:
inputs = keras.Input(shape=(None,), dtype="int64")

# GloVe 단어 임베딩 활용
embedded = embedding_layer(inputs)

x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.keras",
                                    save_best_only=True)
]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)

model = keras.models.load_model("glove_embeddings_sequence_model.keras")

print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 100)         2000000   
                                                                 
 bidirectional_3 (Bidirectio  (None, 64)               34048     
 nal)                                                            
                                                                 
 dropout_6 (Dropout)         (None, 64)                0         
                                                                 
 dense_9 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2,034,113
Trainable params: 34,113
Non-trainable params: 2,000,000
____________________________________________